In [91]:
import pandas as pd   # Para manipulación de datos
import numpy as np    # Para operaciones numéricas
import matplotlib.pyplot as plt  # Para gráficos
import seaborn as sns  # Para gráficos más avanzados

In [92]:
from google.colab import files
uploaded = files.upload()  # Seleccionas el archivo desde tu PC

# Cargar el archivo con codificación compatible
df = pd.read_csv("combate_resultados3.csv", sep=";", encoding="latin-1")

Saving combate_resultados3.csv to combate_resultados3 (3).csv


In [95]:
# Variables dependientes Calculadas

# Contar cuántas veces cada jugador ganó o perdió
resumen = df.groupby(['participante', 'resultado']).size().unstack(fill_value=0)

# TOTAL PARTIDAS
# Agregar el total de partidas
resumen['total_partidas'] = resumen.sum(axis=1)

# TOTAL PARTIDAS GANADAS Y PERDIDAS
# Reordenar las columnas por claridad
resumen = resumen.reindex(columns=['GANO', 'PERDIO', 'total_partidas'], fill_value=0)

# Reordenar las columnas por claridad
resumen = resumen.reindex(columns=['GANO', 'PERDIO', 'total_partidas'], fill_value=0)

# RENDIMIENTO PROMEDIO
# Calcular rendimiento promedio (porcentaje de victorias)
resumen['rendimiento_promedio'] = (resumen['GANO'] / resumen['total_partidas']) * 100

# Redondear a 2 decimales para mayor claridad
resumen['rendimiento_promedio'] = resumen['rendimiento_promedio'].round(2)

# PUNTAJE POR GANANCIA
# Calcular puntaje por ganancia
resumen['puntaje_por_ganancia'] = resumen['GANO'] * 0.7
resumen['puntaje_por_ganancia'] = resumen['puntaje_por_ganancia'].round(3)

# TORNEOS GANADOS X PARTICIPANTE
# --- Contar torneos ganados ---
torneos_ganados = (
    df[(df['resultado'] == 'GANO') & (df['instancia_torneo'].isin(['FJ', 'FG']))]
    .groupby('participante')
    .size()
    .rename('torneos_ganados')
)

# EVENTOS GANADOS X PARTICIPANTE
# --- Contar eventos ganados ---
eventos_ganados = (
    df[(df['resultado'] == 'GANO') & (df['instancia_torneo'].isin(['FT', 'FG']))]
    .groupby('participante')
    .size()
    .rename('eventos_ganados')
)

# --- Unir ambos conteos al resumen ---
resumen = resumen.join([torneos_ganados, eventos_ganados])

# --- Rellenar valores faltantes con 0 ---
resumen[['torneos_ganados', 'eventos_ganados']] = resumen[['torneos_ganados', 'eventos_ganados']].fillna(0).astype(int)

# PUNTAJE RANKING
# --- Calcular el puntaje para ranking ---
resumen['puntaje_ranking'] = (
    (resumen['rendimiento_promedio'] * 0.65) +
    (resumen['puntaje_por_ganancia'] * 0.35) +
    (resumen['torneos_ganados'] * 1) +
    (resumen['eventos_ganados'] * 5)
).round(2)

# --- Mostrar resultado final ---
resumen

,GANO,PERDIO,total_partidas,rendimiento_promedio,puntaje_por_ganancia,torneos_ganados,eventos_ganados,puntaje_ranking
participante,,,,,,,,
Alan Sanchez,0,4,4,0.00,0.0,0,0,0.00
Alberto Cadena,61,101,162,37.65,42.7,4,1,48.42
Alejandro Granda,1,2,3,33.33,0.7,0,0,21.91
Alejandro Matute,1,2,3,33.33,0.7,0,0,21.91
Alejandro Pozo,1,2,3,33.33,0.7,0,0,21.91
...,...,...,...,...,...,...,...,...
Victor Salaz,1,2,3,33.33,0.7,0,0,21.91
William Almachi,1,2,3,33.33,0.7,0,0,21.91
Xavier Calderon,0,4,4,0.00,0.0,0,0,0.00


In [94]:
# Total de partidas por franquicia usando combates únicos
total_partidas_franquicia = (
    df[['franquicia', 'idcombate']]
    .drop_duplicates()        # elimina combates repetidos
    .groupby('franquicia')
    .size()
    .reset_index(name='total_partidas_franquicia')
)

# Número de participantes únicos por franquicia
participantes_franquicia = (
    df.groupby('franquicia')['participante']
    .nunique()
    .reset_index(name='total_participantes_franquicia')
)

# Unir total de partidas + total de participantes
resumen_franquicia = total_partidas_franquicia.merge(
    participantes_franquicia, on='franquicia'
)

resumen_franquicia


,franquicia,total_partidas_franquicia,total_participantes_franquicia
0,BEAT SABER,10,11
1,BLAZBLUE,22,12
2,BLOODY ROAR,60,19
3,CALL OF DUTY,58,36
4,CATHERINE,42,22
5,CRASH BANDICOOT,22,23
6,DEAD OR ALIVE,49,29
7,DRAGON BALL,255,123
8,DUCK HUNT,13,12
9,FIFA,62,64


In [96]:
from IPython.display import HTML, display

# ================================
# 🔧 Preparar datos
# ================================

# Asegurarte de que la columna 'fotourl' exista en df
foto_df = df[['participante', 'fotourl']].drop_duplicates().set_index('participante')

# Si 'fotourl' ya existe en resumen, eliminarla para evitar duplicados
if 'fotourl' in resumen.columns:
    resumen = resumen.drop(columns=['fotourl'])

# Unir fotos al resumen
resumen = resumen.join(foto_df)

# Ordenar alfabéticamente por participante
resumen = resumen.sort_index()

# ================================
# 🎨 Mostrar dashboard visual
# ================================
display(HTML("<h2 style='color:#0d6efd;'>📊 Dashboard de Rendimiento de Participantes</h2>"))

for jugador, row in resumen.iterrows():
    # Asegurar valores nulos manejables
    foto = row.get('fotourl', '')
    if pd.isna(foto):
        foto = ''

    display(HTML(f"""
        <div style="
            display:flex;
            align-items:center;
            gap:15px;
            margin-bottom:18px;
            background:#f8f9fa;
            padding:12px;
            border-radius:12px;
            box-shadow:0 2px 4px rgba(0,0,0,0.1);
        ">
            <img src="{foto}" width="80" style="border-radius:50%; border:2px solid #0d6efd;">
            <div style="line-height:1.6;">
                <b style="font-size:16px; color:#0d6efd;">{jugador}</b><br>
                🏆 <b>Ganó:</b> {row['GANO']} &nbsp;&nbsp;
                ❌ <b>Perdió:</b> {row['PERDIO']} &nbsp;&nbsp;
                🎮 <b>Total:</b> {row['total_partidas']}<br>
                📈 <b>Rendimiento promedio:</b> {row['rendimiento_promedio']:.2f}%<br>
                💰 <b>Puntaje por ganancia:</b> {row['puntaje_por_ganancia']:.2f}<br>
                🥇 <b>Torneos ganados:</b> {row['torneos_ganados']} &nbsp;&nbsp;
                🏅 <b>Eventos ganados:</b> {row['eventos_ganados']}<br>
                ⭐ <b>Puntaje ranking:</b> {row['puntaje_ranking']:.2f}
            </div>
        </div>
    """))


In [97]:
from IPython.display import HTML, display

# ================================
# 🔧 Preparar datos
# ================================
foto_df = df[['participante', 'fotourl']].drop_duplicates().set_index('participante')

# Evitar duplicado de columna 'fotourl'
if 'fotourl' in resumen.columns:
    resumen = resumen.drop(columns=['fotourl'])

# Unir y ordenar
resumen = resumen.join(foto_df)
resumen = resumen.sort_index()

# ================================
# 💅 CSS: Estilo tipo gamer
# ================================
estilos = """
<style>
    body {
        background-color: #0e0e10;
        color: #e0e0e0;
        font-family: 'Roboto Mono', monospace;
    }
    .dashboard-title {
        color: #00ffff;
        font-size: 24px;
        font-weight: bold;
        margin-bottom: 25px;
        text-shadow: 0 0 10px #00ffff;
    }
    .player-card {
        display: flex;
        align-items: center;
        gap: 18px;
        margin-bottom: 18px;
        background: linear-gradient(135deg, #1a1a1d, #2a2a30);
        padding: 16px 20px;
        border-radius: 14px;
        border: 1px solid #00ffff44;
        box-shadow: 0 0 10px #00ffff33;
        transition: all 0.3s ease;
    }
    .player-card:hover {
        transform: scale(1.02);
        box-shadow: 0 0 15px #00ffff99;
        border-color: #00ffffcc;
    }
    .player-card img {
        border-radius: 50%;
        width: 85px;
        height: 85px;
        object-fit: cover;
        border: 2px solid #00ffff;
        box-shadow: 0 0 10px #00ffff66;
    }
    .player-info {
        line-height: 1.6;
    }
    .player-name {
        font-size: 18px;
        font-weight: bold;
        color: #00ffff;
        text-shadow: 0 0 8px #00ffff;
        margin-bottom: 5px;
    }
    .metric {
        font-size: 14px;
        color: #cfcfcf;
    }
    .metric b {
        color: #00ffff;
    }
</style>
"""

# ================================
# 💎 Mostrar dashboard
# ================================
html_output = "<div class='dashboard-title'>⚡ Ranking de Jugadores FURIA A-Z</div>"

for jugador, row in resumen.iterrows():
    foto = row.get('fotourl', '')
    if pd.isna(foto):
        foto = ''
    html_output += f"""
        <div class="player-card">
            <img src="{foto}" alt="foto">
            <div class="player-info">
                <div class="player-name">{jugador}</div>
                <div class="metric">🏆 <b>Ganó:</b> {row['GANO']} &nbsp;&nbsp; ❌ <b>Perdió:</b> {row['PERDIO']} &nbsp;&nbsp; 🎮 <b>Total:</b> {row['total_partidas']}</div>
                <div class="metric">📈 <b>Rendimiento:</b> {row['rendimiento_promedio']:.2f}% &nbsp;&nbsp; 💰 <b>Puntaje:</b> {row['puntaje_por_ganancia']:.2f}</div>
                <div class="metric">🥇 <b>Torneos:</b> {row['torneos_ganados']} &nbsp;&nbsp; 🏅 <b>Eventos:</b> {row['eventos_ganados']}</div>
                <div class="metric">⭐ <b>Ranking:</b> {row['puntaje_ranking']:.2f}</div>
            </div>
        </div>
    """

display(HTML(estilos + html_output))


In [98]:
from IPython.display import HTML, display
import ipywidgets as widgets
import pandas as pd

# ================================
# 🔧 Preparar datos
# ================================
foto_df = df[['participante', 'fotourl']].drop_duplicates().set_index('participante')

# Evitar duplicado de columna 'fotourl'
if 'fotourl' in resumen.columns:
    resumen = resumen.drop(columns=['fotourl'])

# Unir y ordenar
resumen = resumen.join(foto_df)
resumen = resumen.sort_index()

# ================================
# 💅 Estilos CSS tipo gamer
# ================================
estilos = """
<style>
    body {
        background-color: #0e0e10;
        color: #e0e0e0;
        font-family: 'Roboto Mono', monospace;
    }
    .dashboard-title {
        color: #00ffff;
        font-size: 22px;
        font-weight: bold;
        margin-bottom: 20px;
        text-shadow: 0 0 10px #00ffff;
    }
    .player-card {
        display: flex;
        align-items: center;
        gap: 18px;
        margin-top: 15px;
        background: linear-gradient(135deg, #1a1a1d, #2a2a30);
        padding: 16px 20px;
        border-radius: 14px;
        border: 1px solid #00ffff44;
        box-shadow: 0 0 10px #00ffff33;
        transition: all 0.3s ease;
    }
    .player-card:hover {
        transform: scale(1.02);
        box-shadow: 0 0 15px #00ffff99;
        border-color: #00ffffcc;
    }
    .player-card img {
        border-radius: 50%;
        width: 85px;
        height: 85px;
        object-fit: cover;
        border: 2px solid #00ffff;
        box-shadow: 0 0 10px #00ffff66;
    }
    .player-info {
        line-height: 1.6;
    }
    .player-name {
        font-size: 18px;
        font-weight: bold;
        color: #00ffff;
        text-shadow: 0 0 8px #00ffff;
        margin-bottom: 5px;
    }
    .metric {
        font-size: 14px;
        color: #cfcfcf;
    }
    .metric b {
        color: #00ffff;
    }
</style>
"""

# ================================
# ⚙️ Función para mostrar jugador
# ================================
def mostrar_participante(nombre):
    row = resumen.loc[nombre]
    foto = row.get('fotourl', '')
    if pd.isna(foto):
        foto = ''

    html = f"""
        {estilos}
        <div class="dashboard-title">⚡ Perfil del Jugador Seleccionado</div>
        <div class="player-card">
            <img src="{foto}" alt="foto">
            <div class="player-info">
                <div class="player-name">{nombre}</div>
                <div class="metric">🏆 <b>Ganó:</b> {row['GANO']} &nbsp;&nbsp; ❌ <b>Perdió:</b> {row['PERDIO']} &nbsp;&nbsp; 🎮 <b>Total:</b> {row['total_partidas']}</div>
                <div class="metric">📈 <b>Rendimiento:</b> {row['rendimiento_promedio']:.2f}% &nbsp;&nbsp; 💰 <b>Puntaje:</b> {row['puntaje_por_ganancia']:.2f}</div>
                <div class="metric">🥇 <b>Torneos:</b> {row['torneos_ganados']} &nbsp;&nbsp; 🏅 <b>Eventos:</b> {row['eventos_ganados']}</div>
                <div class="metric">⭐ <b>Ranking:</b> {row['puntaje_ranking']:.2f}</div>
            </div>
        </div>
    """
    display(HTML(html))

# ================================
# 🧩 Crear menú desplegable
# ================================
menu = widgets.Dropdown(
    options=sorted(resumen.index.tolist()),
    description='👤 Participante:',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='50%')
)

# Conectar el menú con la función
widgets.interact(mostrar_participante, nombre=menu)


interactive(children=(Dropdown(description='👤 Participante:', layout=Layout(width='50%'), options=('Alan Sanch…

<function __main__.mostrar_participante(nombre)>

In [102]:
from IPython.display import HTML, display
import ipywidgets as widgets
import pandas as pd

# ================================
# 🔧 1. Calcular variables por franquicia
# ================================
resumen_franquicia = (
    df.groupby(['franquicia', 'participante', 'resultado'])
    .size()
    .unstack(fill_value=0)
)

for col in ['GANO', 'PERDIO']:
    if col not in resumen_franquicia.columns:
        resumen_franquicia[col] = 0

resumen_franquicia['total_partidas'] = resumen_franquicia['GANO'] + resumen_franquicia['PERDIO']

resumen_franquicia['rendimiento_promedio'] = (
    resumen_franquicia['GANO'] / resumen_franquicia['total_partidas'].replace(0, 1)
) * 100

torneos_franquicia = (
    df[(df['resultado'] == 'GANO') & (df['instancia_torneo'].isin(['FJ', 'FG']))]
    .groupby(['franquicia', 'participante'])
    .size()
    .rename('torneos_ganados')
)
resumen_franquicia = resumen_franquicia.join(torneos_franquicia, how='left').fillna({'torneos_ganados': 0})
resumen_franquicia['torneos_ganados'] = resumen_franquicia['torneos_ganados'].astype(int)

resumen_franquicia['ranking_franquicia'] = (
    (resumen_franquicia['rendimiento_promedio'] * 0.7) +
    (resumen_franquicia['GANO'] * 0.3) +
    (resumen_franquicia['torneos_ganados'] * 2.5)
).round(2)

# ================================
# 🔗 2. Agregar fotos y logos
# ================================
foto_df = df[['participante', 'fotourl']].drop_duplicates().set_index('participante')
resumen_franquicia = resumen_franquicia.join(foto_df, on='participante')

logo_franquicia = df[['franquicia', 'url_franquicia']].drop_duplicates().set_index('franquicia')

# ================================
# 🎨 3. Estilos CSS mejorados
# ================================
estilos = """
<style>
    body {
        background-color: #0e0e10;
        color: #e0e0e0;
        font-family: 'Roboto Mono', monospace;
    }
    .dashboard-title {
        color: #00ffff;
        font-size: 22px;
        font-weight: bold;
        margin-bottom: 20px;
        text-shadow: 0 0 10px #00ffff;
    }
    .logo-f {
        display: block;
        margin-bottom: 12px;
        width: 150px;
        height: 150px;
        object-fit: contain;
        border-radius: 14px;
        box-shadow: 0 0 18px #00ffff88;
        border: 2px solid #00ffff55;
    }
    .franchise-title {
        color: #bb86fc;
        font-size: 18px;
        margin: 10px 0 15px 0;
        text-shadow: 0 0 6px #bb86fc;
    }
    .player-card {
        display: flex;
        align-items: center;
        gap: 18px;
        margin-top: 12px;
        background: linear-gradient(135deg, #1a1a1d, #2a2a30);
        padding: 16px 20px;
        border-radius: 14px;
        border: 1px solid #00ffff44;
        box-shadow: 0 0 10px #00ffff33;
        transition: all 0.3s ease;
    }
    .player-card:hover {
        transform: scale(1.02);
        box-shadow: 0 0 15px #00ffff99;
        border-color: #00ffffcc;
    }
    .player-card img {
        border-radius: 50%;
        width: 85px;
        height: 85px;
        object-fit: cover;
        border: 2px solid #00ffff;
        box-shadow: 0 0 10px #00ffff66;
    }
    .player-info {
        line-height: 1.6;
    }
    .player-name {
        font-size: 18px;
        font-weight: bold;
        color: #00ffff;
        text-shadow: 0 0 8px #00ffff;
        margin-bottom: 5px;
    }
    .metric {
        font-size: 14px;
        color: #cfcfcf;
    }
    .metric b {
        color: #00ffff;
    }
</style>
"""

# ================================
# ⚙️ 4. Función para mostrar franquicia
# ================================
def mostrar_franquicia(franquicia):
    subset = resumen_franquicia.loc[franquicia].sort_values(by='ranking_franquicia', ascending=False)
    logo_url = logo_franquicia.loc[franquicia, 'url_franquicia'] if franquicia in logo_franquicia.index else ''

    html = f"""
        {estilos}
        <div class="dashboard-title">🎮 Dashboard por Franquicia</div>
        <img src="{logo_url}" alt="logo franquicia" class="logo-f">
        <div class="franchise-title">🏷️ {franquicia}</div>
    """

    for jugador, row in subset.iterrows():
        foto = row.get('fotourl', '')
        if pd.isna(foto): foto = ''
        html += f"""
            <div class="player-card">
                <img src="{foto}" alt="foto">
                <div class="player-info">
                    <div class="player-name">{jugador}</div>
                    <div class="metric">🏆 <b>Ganó:</b> {row['GANO']} &nbsp;&nbsp; ❌ <b>Perdió:</b> {row['PERDIO']} &nbsp;&nbsp; 🎮 <b>Total:</b> {row['total_partidas']}</div>
                    <div class="metric">📈 <b>Rendimiento:</b> {row['rendimiento_promedio']:.2f}% &nbsp;&nbsp; 🏅 <b>Torneos:</b> {row['torneos_ganados']}</div>
                    <div class="metric">⭐ <b>Ranking Franquicia:</b> {row['ranking_franquicia']:.2f}</div>
                </div>
            </div>
        """
    display(HTML(html))

# ================================
# 🧩 5. Menú desplegable
# ================================
menu_franquicia = widgets.Dropdown(
    options=sorted(resumen_franquicia.index.get_level_values('franquicia').unique()),
    description='🎮 Franquicia:',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='50%')
)

widgets.interact(mostrar_franquicia, franquicia=menu_franquicia)


interactive(children=(Dropdown(description='🎮 Franquicia:', layout=Layout(width='50%'), options=('BEAT SABER',…

<function __main__.mostrar_franquicia(franquicia)>

In [100]:
from IPython.display import HTML, display

# ================================
# 🔧 Preparar datos
# ================================
foto_df = df[['participante', 'fotourl']].drop_duplicates().set_index('participante')

# Evitar duplicado de columna 'fotourl'
if 'fotourl' in resumen.columns:
    resumen = resumen.drop(columns=['fotourl'])

# Unir y ordenar por puntaje_ranking descendente
resumen = resumen.join(foto_df)
resumen = resumen.sort_values(by='puntaje_ranking', ascending=False).head(32)  # Top 32

# ================================
# 💅 CSS: Estilo tipo gamer dark
# ================================
estilos = """
<style>
    body {
        background-color: #0e0e10;
        color: #e0e0e0;
        font-family: 'Roboto Mono', monospace;
    }
    .dashboard-title {
        color: #00ffff;
        font-size: 24px;
        font-weight: bold;
        margin-bottom: 25px;
        text-shadow: 0 0 10px #00ffff;
    }
    .player-card {
        display: flex;
        align-items: center;
        gap: 18px;
        margin-bottom: 18px;
        background: linear-gradient(135deg, #1a1a1d, #2a2a30);
        padding: 16px 20px;
        border-radius: 14px;
        border: 1px solid #00ffff44;
        box-shadow: 0 0 10px #00ffff33;
        transition: all 0.3s ease;
    }
    .player-card:hover {
        transform: scale(1.02);
        box-shadow: 0 0 15px #00ffff99;
        border-color: #00ffffcc;
    }
    .top-label {
        font-size: 18px;
        color: #ffea00;
        font-weight: bold;
        text-shadow: 0 0 10px #ffea00;
        width: 80px;
        text-align: center;
    }
    .player-card img {
        border-radius: 50%;
        width: 85px;
        height: 85px;
        object-fit: cover;
        border: 2px solid #00ffff;
        box-shadow: 0 0 10px #00ffff66;
    }
    .player-info {
        line-height: 1.6;
    }
    .player-name {
        font-size: 18px;
        font-weight: bold;
        color: #00ffff;
        text-shadow: 0 0 8px #00ffff;
        margin-bottom: 5px;
    }
    .metric {
        font-size: 14px;
        color: #cfcfcf;
    }
    .metric b {
        color: #00ffff;
    }
</style>
"""

# ================================
# 💎 Mostrar dashboard Top 32
# ================================
html_output = "<div class='dashboard-title'>🏆 Top 32 Global de Jugadores</div>"

for i, (jugador, row) in enumerate(resumen.iterrows(), start=1):
    foto = row.get('fotourl', '')
    if pd.isna(foto):
        foto = ''
    html_output += f"""
        <div class="player-card">
            <div class="top-label">Top {i}</div>
            <img src="{foto}" alt="foto">
            <div class="player-info">
                <div class="player-name">{jugador}</div>
                <div class="metric">🏆 <b>Ganó:</b> {row['GANO']} &nbsp;&nbsp; ❌ <b>Perdió:</b> {row['PERDIO']} &nbsp;&nbsp; 🎮 <b>Total:</b> {row['total_partidas']}</div>
                <div class="metric">📈 <b>Rendimiento:</b> {row['rendimiento_promedio']:.2f}% &nbsp;&nbsp; 💰 <b>Puntaje:</b> {row['puntaje_por_ganancia']:.2f}</div>
                <div class="metric">🥇 <b>Torneos:</b> {row['torneos_ganados']} &nbsp;&nbsp; 🏅 <b>Eventos:</b> {row['eventos_ganados']}</div>
                <div class="metric">⭐ <b>Ranking:</b> {row['puntaje_ranking']:.2f}</div>
            </div>
        </div>
    """

display(HTML(estilos + html_output))


In [101]:
from IPython.display import HTML, display

# ================================
# 🔧 1. Calcular total de partidas por franquicia
# ================================
# Contar combates únicos por franquicia
total_partidas_franquicia = (
    df[['franquicia', 'idcombate']]
    .drop_duplicates()
    .groupby('franquicia')
    .size()
    .reset_index(name='total_partidas_franquicia')
)

# Unir con el logo de la franquicia
logos = df[['franquicia', 'url_franquicia']].drop_duplicates('franquicia')
top_franquicias = total_partidas_franquicia.merge(logos, on='franquicia', how='left')

# Seleccionar Top 6
top_franquicias = top_franquicias.sort_values(by='total_partidas_franquicia', ascending=False).head(6).reset_index(drop=True)

# ================================
# 💅 2. Estilo gamer dark
# ================================
estilos = """
<style>
    body {
        background-color: #0e0e10;
        color: #e0e0e0;
        font-family: 'Roboto Mono', monospace;
    }
    .dashboard-title {
        color: #00ffff;
        font-size: 24px;
        font-weight: bold;
        margin-bottom: 25px;
        text-shadow: 0 0 10px #00ffff;
    }
    .franchise-card {
        display: flex;
        align-items: center;
        gap: 18px;
        margin-bottom: 18px;
        background: linear-gradient(135deg, #1a1a1d, #2a2a30);
        padding: 16px 20px;
        border-radius: 14px;
        border: 1px solid #00ffff44;
        box-shadow: 0 0 10px #00ffff33;
        transition: all 0.3s ease;
    }
    .franchise-card:hover {
        transform: scale(1.03);
        box-shadow: 0 0 20px #00ffffaa;
        border-color: #00ffffcc;
    }
    .top-label {
        font-size: 18px;
        color: #ffea00;
        font-weight: bold;
        text-shadow: 0 0 10px #ffea00;
        width: 80px;
        text-align: center;
    }
    .franchise-card img {
        border-radius: 10px;
        width: 100px;
        height: 100px;
        object-fit: contain;
        background-color: #ffffff22;
        padding: 8px;
        box-shadow: 0 0 15px #00ffff66;
    }
    .franchise-info {
        line-height: 1.6;
    }
    .franchise-name {
        font-size: 20px;
        font-weight: bold;
        color: #00ffff;
        text-shadow: 0 0 8px #00ffff;
        margin-bottom: 5px;
    }
    .metric {
        font-size: 15px;
        color: #cfcfcf;
    }
    .metric b {
        color: #00ffff;
    }
</style>
"""

# ================================
# 💎 3. Renderizar Dashboard
# ================================
html_output = "<div class='dashboard-title'>🔥 Top 6 Franquicias Más Jugadas</div>"

for i, row in top_franquicias.iterrows():
    logo = row.get('url_franquicia', '')
    if pd.isna(logo):
        logo = ''
    html_output += f"""
        <div class="franchise-card">
            <div class="top-label">Top {i+1}</div>
            <img src="{logo}" alt="logo">
            <div class="franchise-info">
                <div class="franchise-name">{row['franquicia']}</div>
                <div class="metric">🎮 <b>Total de Partidas:</b> {row['total_partidas_franquicia']}</div>
            </div>
        </div>
    """

display(HTML(estilos + html_output))


In [88]:
import plotly.graph_objects as go
import pandas as pd

# ================================
# 🔧 1. Preparar datos Top 10
# ================================
top10 = resumen.sort_values(by='puntaje_ranking', ascending=False).head(10).reset_index()

# ================================
# 🎨 2. Crear gráfico base
# ================================
fig = go.Figure()

# Agregar las burbujas (transparentes, solo para ubicar)
fig.add_trace(go.Scatter(
    x=top10['rendimiento_promedio'],
    y=top10['GANO'],
    text=top10['participante'],
    mode='markers+text',
    textposition='top center',
    marker=dict(
        size=top10['puntaje_ranking'] * 1.5,  # Escalado del tamaño
        color='rgba(0, 255, 255, 0.3)',       # Cian translúcido
        line=dict(width=2, color='rgba(0, 255, 255, 0.6)')
    ),
    hovertemplate=(
        "<b>%{text}</b><br>" +
        "🏆 Ganadas: %{y}<br>" +
        "📈 Rendimiento: %{x:.2f}%<br>" +
        "⭐ Puntaje: %{marker.size:.2f}"
    )
))

# ================================
# 🖼️ 3. Agregar imágenes en las burbujas
# ================================
for i, row in top10.iterrows():
    fig.add_layout_image(
        dict(
            source=row['fotourl'],
            x=row['rendimiento_promedio'],
            y=row['GANO'],
            xref="x",
            yref="y",
            sizex=5,  # controla el tamaño de la imagen
            sizey=5,
            xanchor="center",
            yanchor="middle",
            layer="above"
        )
    )

# ================================
# 💅 4. Estilo del gráfico
# ================================
fig.update_layout(
    title=dict(
        text="🔥 TOP 10 PARTICIPANTES - GRÁFICO DE BURBUJAS",
        font=dict(size=22, color="#00ffff"),
        x=0.5
    ),
    xaxis=dict(
        title="📈 Rendimiento Promedio (%)",
        gridcolor="#222",
        color="#00ffff"
    ),
    yaxis=dict(
        title="🏆 Partidas Ganadas",
        gridcolor="#222",
        color="#00ffff"
    ),
    plot_bgcolor="#0e0e10",
    paper_bgcolor="#0e0e10",
    font=dict(family="Roboto Mono", color="#e0e0e0"),
    hoverlabel=dict(bgcolor="#1a1a1d", font_size=12, font_family="Roboto Mono")
)

fig.show()


In [89]:
import plotly.graph_objects as go
import pandas as pd

# ================================
# 🔧 1. Filtrar jugadores con torneos ganados
# ================================
jugadores_campeones = resumen[resumen['torneos_ganados'] > 0].sort_values(
    by='torneos_ganados', ascending=False
)

# Si no hay jugadores con campeonatos
if jugadores_campeones.empty:
    print("⚠️ No hay jugadores con torneos ganados registrados.")
else:
    # ================================
    # 🎨 2. Crear gráfico tipo donut
    # ================================
    fig = go.Figure(
        data=[
            go.Pie(
                labels=jugadores_campeones.index,
                values=jugadores_campeones['torneos_ganados'],
                hole=0.55,
                marker=dict(
                    colors=[
                        "#00ffff", "#bb86fc", "#03dac6", "#ff0266",
                        "#ffab00", "#6200ee", "#3700b3", "#03a9f4",
                        "#18ffff", "#a7ffeb"
                    ],
                    line=dict(color="#0e0e10", width=2)
                ),
                hovertemplate="<b>%{label}</b><br>🏆 Torneos ganados: %{value}<extra></extra>",
                textinfo="label+value",
                textfont=dict(size=14, color="#e0e0e0")
            )
        ]
    )

    # ================================
    # 💅 3. Personalizar el diseño visual
    # ================================
    fig.update_layout(
        title=dict(
            text="🏆 TOP JUGADORES CON CAMPEONATOS",
            font=dict(size=24, color="#00ffff", family="Roboto Mono"),
            x=0.5
        ),
        paper_bgcolor="#0e0e10",
        plot_bgcolor="#0e0e10",
        font=dict(color="#e0e0e0", family="Roboto Mono"),
        showlegend=False,
        annotations=[
            dict(
                text="Torneos Ganados",
                x=0.5, y=0.5,
                font=dict(size=16, color="#00ffff"),
                showarrow=False
            )
        ]
    )

    fig.show()
